<a href="https://colab.research.google.com/github/acts701/LLM/blob/main/250222_%EA%B0%9C%EB%B0%9C%EC%9D%BC%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine tunning Model 생성 후 새로운 접근

|방식|기존 Fine-Tuning|새로운 RAG + Re-Ranker|
|---|-------------|-------------|
|정확도|낮음 (일반화 어려움)|높음 (Re-Ranker 적용)|
|검색 방식|FAISS만 사용|FAISS + Re-Ranker|
|모델 활용|Fine-Tuning된 GPT-3.5 사용|GPT-4 직접 사용|
|실시간 반영|❌ Fine-Tuning 필요|✅ 문서 변경 즉시 반영 가능|
|추가 학습 |필요	필요 (JSONL 추가)|필요 없음 (문서 업데이트 가능)|

# 경과
- 결론적으로 RAG랑 re-ranker를 쓰더라도 정확도는 처참하다.
```python
query = "22장에서 예루살렘을 무엇이라 부르지?"
response = process_query(query)
print(f"🔹 최종 답변: {response}")
🔹 최종 답변: 이사야 22장 1절에서 예루살렘을 "바람계곡"이라고 부르고 있습니다.
```
- 이사야서에 '바람계곡'이라는 표현은 나온 적이 없는데 이런 식으로 답변이 생성되고 있다.
``` bash
query = "이사야서에서 그루터기의 의미는?"
🔹 최종 답변: 이사야서에서 그루터기는 하나님의 은혜로써 우리에게 속한 것을 나타냅니다. 그루터기는 옛 굴을 쓸어버리고 새로운 것으로 채우는 작업을 의미하며, 하나님의 은혜와 축복을 통해 새로운 삶과 복음이 우리에게 주어진다는 의미를 갖고 있습니다. 이를 통해 하나님의 구원이 우리 삶에 새로운 행복과 축복을 가져다 준다는 것을 알 수 있습니다.
```
- 이 답변 역시 의미가 다른 것은 물론이고 갑자기 굴이 왜 나오는지 당황스럽게 생성된다.

# 한계
- 아무래도 embedding vector로 검색이 들어가는 방식이 문제인 것 같다.
- 기술문서처럼 정확한 워딩을 찾는 것이 아니라 전체 문장에서 의미론적인 결과를 추론해야 하는 방식이기에 embedding vector가 맞지 않는 것일까?

# leverage
- 최후의 수단 정도로 생각했던 GPT에 file upload 시켜두고 질문 들어오면 file 찾아서 toss 하는 방식을 써보자.
- 뭐라도 되는걸 먼저 확인할 목적.
- 문제는 100개의 원본 파일 중 사용자의 질문이 어느 파일에서 나올지 모른다는 것.
- vector store에 upload 해두고 id만 가지고 있으면 된다길래 token 태우면서 해봤는데 역시나 session이 끊기지도 않았는데 file은 다 사라지고 해당 vector store에 그런 파일 없단다;;
``` python
vector_store_files = client.beta.vector_stores.files.list(vector_store_id=vector_store_id)
file_list = [file.id for file in vector_store_files.data]
print("🔹 Vector Store 내 포함된 파일 ID 목록:", file_list)
 Vector Store 내 포함된 파일 ID 목록: []
```
